In [2]:
import os
from copy import deepcopy
from random import randrange
from functools import partial
import torch
import accelerate
import bitsandbytes as bnb
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)

import wandb

/home//miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: berkgungor-de. Use `wandb login --relogin` to force relogin


True

In [3]:
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
new_model = "zephyr-Golf-Instruct-beta_v2"

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)


model.config.use_cache = False
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]


In [5]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    # lm_head is often excluded.
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)


lora_alpha = 16
lora_dropout = 0.1
lora_r = 8


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=modules,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, peft_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 20971520 | total: 7262703616 | Percentage: 0.2888%


In [6]:
### LOAD DATASET

dataset_dir = "zephyr_converted_data.json"
dataset = load_dataset('json', data_files=dataset_dir, split='train')

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def format_prompt(sample):

    INTRO = "<|system|>You are a Golf Coach. Your job is to help your user to improve their  skills with providing exercises and training plans."
    END = "Instruction: Provide  exercises or training plans based on user's request or write a response appropriate to the conversation."

    conversations = ""
    for response in sample["conversations"]:
      from_, value = response["from"], response["value"]
      conversations += f"<{from_}>: " + value + "\n"

    sample["text"] = "\n\n".join([INTRO, conversations])
    return sample


def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, dataset: str, seed: int = 42):
    # Format each prompt.
    print("Preprocessing dataset...")
    dataset = dataset.map(format_prompt)

    def preprocess_batch(batch, tokenizer, max_length):
        return tokenizer(
            batch["text"],
            max_length=max_length,
            truncation=True,
        )


    # Apply preprocessing to each batch of the dataset & and remove "conversations" and "text" fields.
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["conversations", "text"],
    )
    # Shuffle dataset.
    dataset = dataset.shuffle(seed=seed)

    return dataset

max_length = 512
dataset = preprocess_dataset(tokenizer, max_length, dataset)

Preprocessing dataset...


Map: 100%|██████████| 1001/1001 [00:00<00:00, 5243.14 examples/s]


In [7]:
#=========================== WANDB ==========================================
run = wandb.init(
    project="finetuning_zephyr7b_beta_v3",
    name="log_dataset",
)

dataset.save_to_disk("GolfInstruct_beta_prep3.hf")
artifact = wandb.Artifact(name="GolfInstruct_beta_prep3", type="dataset")
artifact.add_dir("./GolfInstruct_beta_prep3.hf", name="train")
run.log_artifact(artifact)

run.finish()

run = wandb.init(
    project="finetuning_zephyr7b_beta_v3",   # Project name.
    name="run0",                     # name of the run within this project.
)
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # Log model checkpoints.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saving the dataset (1/1 shards): 100%|██████████| 1001/1001 [00:00<00:00, 90259.23 examples/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Adding directory to artifact (./GolfInstruct_beta_prep3.hf)... Done. 0.0s


In [8]:
#=========================== TRAINING ==========================================
training_args = TrainingArguments(
    output_dir="ZEPHYR_outputs_beta_v3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_grad_norm=1.0,
    max_steps=2000,
    lr_scheduler_type="linear",
    warmup_steps=5,
    fp16=True,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=500,
    optim="paged_adamw_8bit",
    report_to="wandb"
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=dataset,
)

In [9]:
results = trainer.train()  # Now we just run train()!
trainer.model.save_pretrained(new_model)
run.finish()

  0%|          | 0/2000 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home//miniconda3/envs/torch/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  0%|          | 1/2000 [00:03<2:10:48,  3.93s/it]

{'loss': 1.8362, 'learning_rate': 4e-05, 'epoch': 0.0}


  0%|          | 2/2000 [00:07<2:01:15,  3.64s/it]

{'loss': 1.9154, 'learning_rate': 8e-05, 'epoch': 0.01}


  0%|          | 3/2000 [00:09<1:40:54,  3.03s/it]

...

{'loss': 0.0691, 'learning_rate': 2.005012531328321e-07, 'epoch': 7.99}


wandb: Adding directory to artifact (./ZEPHYR_outputs_beta_v3/checkpoint-2000)... Done. 0.5s
100%|██████████| 2000/2000 [1:36:33<00:00,  2.90s/it]

{'train_runtime': 5793.7094, 'train_samples_per_second': 1.381, 'train_steps_per_second': 0.345, 'train_loss': 0.2699704036815092, 'epoch': 7.99}


100%|██████████| 2000/2000 [1:36:34<00:00,  2.90s/it]


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▇▇█▇▆▇▄▃▄▄▂▄▅▂▂▃▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,7.99
train/global_step,2000


In [4]:
### Merge model

def merge_model():
    # Merge the model with LoRA weights
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    merged_model= PeftModel.from_pretrained(base_model, new_model)
    merged_model= merged_model.merge_and_unload()
    torch.cuda.empty_cache()
    # Save the merged model
    merged_model.save_pretrained("zephyr_beta_merged_model",safe_serialization=True)
    tokenizer.save_pretrained("zephyr_beta_merged_model")
    print("========= merged Model saved =========")
    return merged_model, tokenizer

merge_model()

Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.04s/it]


========= merged Model saved =========


(MistralForCausalLM(
   (model): MistralModel(
     (embed_tokens): Embedding(32000, 4096, padding_idx=2)
     (layers): ModuleList(
       (0-31): 32 x MistralDecoderLayer(
         (self_attn): MistralAttention(
           (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
           (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
           (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
           (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
           (rotary_emb): MistralRotaryEmbedding()
         )
         (mlp): MistralMLP(
           (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
           (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
           (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
           (act_fn): SiLUActivation()
         )
         (input_layernorm): MistralRMSNorm()
         (post_attention_layernorm): MistralRMSNorm()
